In [ ]:
!pip install timm
!pip install monai
!pip install ray

In [ ]:
import os
from tqdm import tqdm
import pickle
import argparse
import time
import torch
import yaml
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss
from IPython.display import display
import numpy as np

from utils import set_seed, load_model, save, get_model, update_optimizer, get_data
from epoch import train_epoch, val_epoch, test_epoch
import cli
import greenstand_utils as gu  # GREENSTAND
from monai.losses import FocalLoss # GREENSTAND
from ray import air, tune # GREENSTAND
from ray.tune.schedulers import ASHAScheduler # GREENSTAND

def load_config_file(hyperparameter_config_file='hyperparameters.yaml'):
    with open(hyperparameter_config_file) as file:
        contents = yaml.safe_load(file)
    return contents

def get_args(contents):
    new_args = []
    for item in contents:
        new_args.append('--' + item)
        new_args.append(str(contents[item]))
    return new_args


def train(args):
    set_seed(args, use_gpu=torch.cuda.is_available())
    
    # Get Data  # GREENSTAND
    g_args = vars(args) # GREENSTAND
    g_args['prefixes'] = g_args['prefixes'].split(',') # GREENSTAND
    train_loader, val_loader, test_loader, dataset_attributes = gu.sync_split_get_dataloaders(vars(args), True)  # GREENSTAND
    print(f"Dataset Attributes: {dataset_attributes}") # GREENSTANDS

    model = gu.load_preloaded_model(args, dataset_attributes)  # GREENSTAND - Get pretrained model if specified 
    
    # GREENSTAND - Choose loss
    if args.use_focal_loss == 'y':
        criteria = FocalLoss(to_onehot_y=True) #BinaryFocalLossWithLogits(alpha=1.0)
    else: 
        criteria = CrossEntropyLoss()

    if args.use_gpu:
        torch.cuda.set_device(0)
        model.cuda()
        criteria.cuda()

    # GREENSTAND - Choose Optimizer
    if args.use_adam_optimizer == 'y':
        optimizer = Adam(model.parameters(), lr=args.lr, weight_decay=args.mu)
    else:
        optimizer = SGD(model.parameters(), lr=args.lr, momentum=0.9, weight_decay=args.mu, nesterov=True)

    # Containers for storing metrics over epochs
    loss_train, acc_train, topk_acc_train = [], [], []
    loss_val, acc_val, topk_acc_val, avgk_acc_val, class_acc_val = [], [], [], [], []

    save_name = args.save_name_xp.strip()
    save_dir = os.path.join(os.getcwd(), 'results', save_name)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    print('args.k : ', args.k)

    lmbda_best_acc = None
    best_val_acc = float('-inf')

    for epoch in tqdm(range(args.n_epochs), desc='epoch', position=0):
        t = time.time()
        optimizer = update_optimizer(optimizer, lr_schedule=dataset_attributes['lr_schedule'], epoch=epoch)

        loss_epoch_train, acc_epoch_train, topk_acc_epoch_train = train_epoch(model, optimizer, train_loader,
                                                                              criteria, loss_train, acc_train,
                                                                              topk_acc_train, args.k,
                                                                              dataset_attributes['n_train'],
                                                                              args.use_gpu)

        loss_epoch_val, acc_epoch_val, topk_acc_epoch_val, \
        avgk_acc_epoch_val, lmbda_val = val_epoch(model, val_loader, criteria,
                                                  loss_val, acc_val, topk_acc_val, avgk_acc_val,
                                                  class_acc_val, args.k, dataset_attributes, args.use_gpu)
        # Send the current training result back to Tune
        print(list(avgk_acc_epoch_val.values())[0])
        tune.report(mean_accuracy=list(avgk_acc_epoch_val.values())[0])

        # save model at every epoch
        save(model, optimizer, epoch, os.path.join(save_dir, save_name + '_weights.tar'))

        # save model with best val accuracy
        if acc_epoch_val > best_val_acc:
            best_val_acc = acc_epoch_val
            lmbda_best_acc = lmbda_val
            save(model, optimizer, epoch, os.path.join(save_dir, save_name + '_weights_best_acc.tar'))

        print()
        print(f'epoch {epoch} took {time.time()-t:.2f}')
        print(f'loss_train : {loss_epoch_train}')
        print(f'loss_val : {loss_epoch_val}')
        print(f'acc_train : {acc_epoch_train} / topk_acc_train : {topk_acc_epoch_train}')
        print(f'acc_val : {acc_epoch_val} / topk_acc_val : {topk_acc_epoch_val} / '
              f'avgk_acc_val : {avgk_acc_epoch_val}')

    # load weights corresponding to best val accuracy and evaluate on test
    load_model(model, os.path.join(save_dir, save_name + '_weights_best_acc.tar'), args.use_gpu)
    loss_test_ba, acc_test_ba, topk_acc_test_ba, \
    avgk_acc_test_ba, class_acc_test, confuse = test_epoch(model, test_loader, criteria, args.k,
                                                  lmbda_best_acc, args.use_gpu,
                                                  dataset_attributes)
    
    print("Average test accuracy: {}".format(avgk_acc_test_ba))
    print("Average class accuracies: {}".format(class_acc_test))
    display(confuse)

    # Save the results as a dictionary and save it as a pickle file in desired location

    results = {'loss_train': loss_train, 'acc_train': acc_train, 'topk_acc_train': topk_acc_train,
               'loss_val': loss_val, 'acc_val': acc_val, 'topk_acc_val': topk_acc_val, 'class_acc_val': class_acc_val,
               'avgk_acc_val': avgk_acc_val,
               'test_results': {'loss': loss_test_ba,
                                'accuracy': acc_test_ba,
                                'topk_accuracy': topk_acc_test_ba,
                                'avgk_accuracy': avgk_acc_test_ba,
                                'class_acc_dict': class_acc_test},
               'params': args.__dict__}

    
    with open(os.path.join(save_dir, save_name + '.pkl'), 'wb') as f:
        pickle.dump(results, f)
        
    return acc_test_ba
        
def predict(args):
    set_seed(args, use_gpu=torch.cuda.is_available())
    
    # Get Data  # GREENSTAND
    g_args = vars(args) # GREENSTAND
    g_args['prefixes'] = g_args['prefixes'].split(',') # GREENSTAND
    train_loader, val_loader, test_loader, dataset_attributes = gu.sync_split_get_dataloaders(vars(args), (g_args['skip_sync']=='y'))  # GREENSTAND
    print(f"Dataset Attributes: {dataset_attributes}") # GREENSTANDS

    model = gu.load_preloaded_model_prediction(args, dataset_attributes)  # GREENSTAND - Get pretrained model if specified 
    
    # GREENSTAND - Choose loss
    if args.use_focal_loss == 'y':
        criteria = FocalLoss(to_onehot_y=True) #BinaryFocalLossWithLogits(alpha=1.0)
    else: 
        criteria = CrossEntropyLoss()

    if args.use_gpu:
        torch.cuda.set_device(0)
        model.cuda()
        criteria.cuda()

    # GREENSTAND - Choose Optimizer
    if args.use_adam_optimizer == 'y':
        optimizer = Adam(model.parameters(), lr=args.lr, weight_decay=args.mu)
    else:
        optimizer = SGD(model.parameters(), lr=args.lr, momentum=0.9, weight_decay=args.mu, nesterov=True)

    # Containers for storing metrics over epochs
    loss_train, acc_train, topk_acc_train = [], [], []
    loss_val, acc_val, topk_acc_val, avgk_acc_val, class_acc_val = [], [], [], [], []

    lmbda_best_acc = None
    best_val_acc = float('-inf')

    # load weights corresponding to best val accuracy and evaluate on test
    loss_test_ba, acc_test_ba, topk_acc_test_ba, \
    avgk_acc_test_ba, class_acc_test, confuse = test_epoch(model, test_loader, criteria, args.k,
                                                  lmbda_best_acc, args.use_gpu,
                                                  dataset_attributes)
    
    print("Average test accuracy: {}".format(avgk_acc_test_ba))
    print("Average class accuracies: {}".format(class_acc_test))
    display(confuse)

    # Save the results as a dictionary and save it as a pickle file in desired location

    results = {'loss_train': loss_train, 'acc_train': acc_train, 'topk_acc_train': topk_acc_train,
               'loss_val': loss_val, 'acc_val': acc_val, 'topk_acc_val': topk_acc_val, 'class_acc_val': class_acc_val,
               'avgk_acc_val': avgk_acc_val,
               'test_results': {'loss': loss_test_ba,
                                'accuracy': acc_test_ba,
                                'topk_accuracy': topk_acc_test_ba,
                                'avgk_accuracy': avgk_acc_test_ba,
                                'class_acc_dict': class_acc_test},
               'params': args.__dict__}

In [ ]:
"""To run: python main.py --lr=0.05 --n_epochs=80 --k 1 3 5 10 --model=resnet50 --root=path_to_data --save_name_xp=xp1"""

"""
Provide your arguments here in this format:
[
 argname1, argvalue1,
 argname2, argvalue2
]
"""
current_dir = os.getcwd()
print(f"Use CUDA: {torch.cuda.is_available()}")

def run(config_replace={}):
    # Get config
    os.chdir(current_dir)
    config = load_config_file()
    for k in config_replace:
        config[k] = config_replace[k] 
        # If resnet, make sure to also switch preloaded model location
        if k == "model" and config_replace[k] == "resnet50":
            config['preloaded_model_location'] = 'model/pretrained_model.pth'
    # Convert to args
    arg_list = get_args(config)
    parser = argparse.ArgumentParser()
    cli.add_all_parsers(parser)
    args = parser.parse_args(args=arg_list)
    # Run 
    if config['train_model'] == 'y':
        train(args)
    else:
        predict(args)
        
def tuner():
    # Define the search space for the hyperparameter
    search_space = {
        "lr": tune.loguniform(1e-4, 1e-1),
        "model" : tune.choice(["inception_v4", "resnet50"]),
        "mu":  tune.choice([0.0000, 0.0001]),
        "use_adam_optimizer" : tune.choice(["y", "n"]),
        "use_focal_loss" : tune.choice(["y", "n"]),
        "transfer_learning_freeze_weights" : tune.choice(["y", "n"])
    }
    # Define the tuner and set early stopper scheduler
    tuner = tune.Tuner(
        tune.with_resources(run, {"cpu": 4,  'gpu': 1}),
        tune_config=tune.TuneConfig(num_samples=20, scheduler=ASHAScheduler(metric="mean_accuracy", mode="max")),
        param_space=search_space,
        
    )
    # Run tuner
    results = tuner.fit()
    # Plot by epoch the trials
    ax = None  # This plots everything on the same plot
    dfs = {result.log_dir: result.metrics_dataframe for result in results}
    for d in dfs.values():
        ax = d.mean_accuracy.plot(ax=ax, legend=False)
    
def main():
    config = load_config_file()
    if config['use_tuner'] == 'y':
        tuner()
    else:
        run()
        
main()